In [1]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPE表の作成@CG

In [2]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [3]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

# 重み付き平均MAPE表の作成@MG

In [4]:
date: str = "2022年8月26日"

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

mg_input_list_problem_size: list[int] = [32, 64, 128, 256]
mg_input_list_nit: list[int] = [4, 10, 20, 35]

mg_target_list_problem_size: list[int] = [512]
mg_target_list_nit: list[int] = [50]

# input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=input_list_process,
#     list_na=cg_input_list_na,
#     list_nonzer=cg_input_list_nonzer,
#     list_niter=cg_input_list_niter,
#     list_shift=cg_input_list_shift,
#     csvDir=csvDirPath,
# )
# input_rawDF_cg = input_rawDF_cg.rename(columns={"Name":"functionName"})

input_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=input_list_process,
    list_problem_size=mg_input_list_problem_size,
    list_nit=mg_input_list_nit,
    csvDir=csvDirPath,
)
input_rawDF_mg = input_rawDF_mg.rename(columns={"Name": "functionName"})


# target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
#     list_process=target_list_process,
#     list_na=cg_target_list_na,
#     list_nonzer=cg_target_list_nonzer,
#     list_niter=cg_target_list_niter,
#     list_shift=cg_target_list_shift,
#     csvDir=csvDirPath,
# )
# target_rawDF_cg = target_rawDF_cg.rename(columns={"Name":"functionName"})

target_rawDF_mg: pd.DataFrame = return_rawDF_mg(
    list_process=target_list_process,
    list_problem_size=mg_target_list_problem_size,
    list_nit=mg_target_list_nit,
    csvDir=csvDirPath,
)
target_rawDF_mg = target_rawDF_mg.rename(columns={"Name": "functionName"})

mg_list_exp: list[str] = ["process", "problem_size", "nit"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_mg["functionName"])))

In [5]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_mg[
        input_rawDF_mg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_mg[
        target_rawDF_mg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=mg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")
    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[mg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_mg: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [6]:
retNum_mg

38.136058981254756

In [7]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


In [8]:
date: str = "2022年8月26日"


input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

targetFunctionNames: list[str] = [
    "VECSET",
    "MPI_Wait()",
    "MPI_Send()",
    "SPRNVC",
    "RANDLC",
    "MPI_Irecv()",
    "ICNVRT",
]

function_names: list[str] = list(set(input_rawDF_cg["Name"].tolist()))

input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})


list_modelName_dict: dict[str, str] = {
    "modelIp": "線形",
    "modelLog": "対数",
    "modelLinAndIp": "線形＆反比例",
    "modelLinAndLog": "線形＆対数",
    "modelIpAndLin": "反比例＆線形",
    "modelIpAndLog": "反比例＆対数",
    "modelLogAndLin": "対数＆線形",
    "modelLogAndIp": "対数＆反比例",
    "modelProcessDividedByProblemSize": "プロセス数を問題サイズで割る",
    "modelProblemSizeDividedByProcess": "問題サイズをプロセス数で割る",
    "modelLinearSumOf2elementCombination": "全変数の組合せ積の総和",
    "modelLinearSumOfElementCombinations": "全変数の組合せ積の線形和",
    "modelLinearSumOf2elementCombinationWithSquared": "2要素の積の線形和（2乗含む）",
    "modelLinearSumOf2elementCombinationWithCubed": "2要素の積の線形和（3乗含む）",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses": "問題サイズの変数の総乗とプロセス数とをかける",
    "modelInfiniteProductOfProblemSizeDividedByProcesses": "問題サイズの変数の総乗をプロセス数で割る",
    "modelSquareRootOfProcess": "プロセス数の平方根",
    "modelSquareRootTimesOtherElems": "プロセス数の平方根と他説明変数の積",
    "modelObeyOneParameter": "一つのパラメータに依存する",
    "modelLin": "線形モデル"
    # "modelBasicTree",
}

/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [9]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_before = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_before

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,modelIpAndLog,modelLogAndLin,modelLogAndIp,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,functionName,最低値,最適モデル
0,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,SPRNVC,3.033609e-14,modelLogAndLin
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,24.184348,3.033609e-14,40.738362,14.024686,2.359154e-12,19.369661,25.924923,VECSET,3.033609e-14,modelLogAndLin
2,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,43.918426,2.550709e+01,63.602600,13.725285,1.353524e+01,26.548026,36.391518,ICNVRT,1.353524e+01,modelLinearSumOfElementCombinations
3,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Irecv(),1.193837e+01,modelLinearSumOfElementCombinations
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Send(),1.193837e+01,modelLinearSumOfElementCombinations
5,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,43.918430,2.550716e+01,63.602559,13.725277,1.353524e+01,26.547929,36.391398,RANDLC,1.353524e+01,modelLinearSumOfElementCombinations
6,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,45.842553,4.102758e+01,55.026591,23.419225,1.193837e+01,26.893264,33.968244,MPI_Wait(),1.193837e+01,modelLinearSumOfElementCombinations


In [10]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [11]:
print(
    resultDF_before.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正前）", siunitx=True)
)

print(f"retNum_cg_before = {retNum_cg_before}")

\begin{table}
\caption{（修正前）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
2 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
3 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
4 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
0 & SPRNVC & 0.000000 & 対数＆線形 \\
1 & VECSET & 0.000000 & 対数＆線形 \\
\end{tabular}
\end{table}

retNum_cg_before = 7.827486063247428


In [12]:
result_series_list: list[pd.DataFrame] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelInfiniteProductOfProblemSizeMultipliedByProcesses",
    "modelInfiniteProductOfProblemSizeDividedByProcesses",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for function_name in function_names:
    if (function_name in targetFunctionNames) == False:
        continue
    input_rawDF_per_function: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == function_name
    ]

    models = Models(
        inputDF=input_rawDF_per_function,
        expVarColNames=cg_list_exp,
        resVarColNames=list_res,
        targetDF=None,
        modelNames=list_modelName,
    )

    models.setUpDataBeforeCalcLr()
    models.calcLr()
    models.calcMAPE()

    dictCalcedMAPE = models.returnCalculatedMAPE()

    for key in dictCalcedMAPE.keys():
        dictCalcedMAPE[key] = float(dictCalcedMAPE[key])

    dict_for_series: dict = copy.deepcopy(dictCalcedMAPE)
    dict_for_series["functionName"] = function_name

    series: pd.Series = pd.Series(dict_for_series)
    result_series_list.append(series)

resultDF: pd.DataFrame = pd.DataFrame(result_series_list)
resultDF = addLowestMAPEColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)
resultDF_after = addLowestMAPEsModelNameColumn(
    inputDF=resultDF, model_name_list=list_modelName, version=2
)

resultDF_after

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


,modelLin,modelIp,modelLog,modelProcessDividedByProblemSize,modelProblemSizeDividedByProcess,modelInfiniteProductOfProblemSizeDividedByProcesses,modelInfiniteProductOfProblemSizeMultipliedByProcesses,modelLinAndIp,modelLinAndLog,modelIpAndLin,...,modelLinearSumOf2elementCombination,modelLinearSumOfElementCombinations,modelLinearSumOf2elementCombinationWithSquared,modelLinearSumOf2elementCombinationWithCubed,modelSquareRootOfProcess,modelSquareRootTimesOtherElems,modelObeyOneParameter,functionName,最低値,最適モデル
0,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,SPRNVC,0.000000,modelObeyOneParameter
1,1.056736e-13,40.738362,24.184347,59.575073,95.488585,58.108038,58.101588,40.738362,24.184347,3.108404e-14,...,14.024686,2.359154e-12,19.369661,25.924923,3.326853e-13,79.407704,0.000000,VECSET,0.000000,modelObeyOneParameter
2,2.550709e+01,63.602611,43.918416,68.457739,68.707184,66.810509,66.800152,63.602603,43.918418,2.550710e+01,...,13.725285,1.353524e+01,26.548026,36.391518,2.550709e+01,110.789284,25.507090,ICNVRT,13.535235,modelLinearSumOfElementCombinations
3,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Irecv(),11.938374,modelLinearSumOfElementCombinations
4,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Send(),11.938374,modelLinearSumOfElementCombinations
5,2.550716e+01,63.602569,43.918421,68.457716,68.703722,66.810487,66.800130,63.602561,43.918423,2.550717e+01,...,13.725277,1.353524e+01,26.547929,36.391398,2.550716e+01,110.789118,25.507161,RANDLC,13.535239,modelLinearSumOfElementCombinations
6,3.425747e+01,52.562966,45.284269,48.827706,381.319302,61.239420,52.016990,57.396654,44.259877,4.179520e+01,...,23.419225,1.193837e+01,26.893264,33.968244,3.761452e+01,63.622956,34.257466,MPI_Wait(),11.938374,modelLinearSumOfElementCombinations


In [13]:
list_series: list[pd.Series] = []

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

In [14]:
print(
    resultDF_after.loc[:, ["functionName", "最低値", "最適モデル"]]
    .sort_values("functionName")
    .rename(columns={"functionName": "関数名", "最低値": "重み付きMAPE"})
    .replace(list_modelName_dict)
    .style.to_latex(label=f"{date}_resultDF_before", caption=f"（修正後）", siunitx=True)
)

print(f"retNum_cg_after = {retNum_cg_after}")

\begin{table}
\caption{（修正後）}
\label{2022年8月26日_resultDF_before}
\begin{tabular}{llSl}
{} & {関数名} & {重み付きMAPE} & {最適モデル} \\
2 & ICNVRT & 13.535235 & 全変数の組合せ積の線形和 \\
3 & MPI_Irecv() & 11.938374 & 全変数の組合せ積の線形和 \\
4 & MPI_Send() & 11.938374 & 全変数の組合せ積の線形和 \\
6 & MPI_Wait() & 11.938374 & 全変数の組合せ積の線形和 \\
5 & RANDLC & 13.535239 & 全変数の組合せ積の線形和 \\
0 & SPRNVC & 0.000000 & 一つのパラメータに依存する \\
1 & VECSET & 0.000000 & 一つのパラメータに依存する \\
\end{tabular}
\end{table}

retNum_cg_after = 7.827486063247428


In [15]:
%reset -f

# jupyter_pwd = %pwd
# if jupyter_pwd == "/":
#     %cd /workspace


# ipynb形式のライブラリのインポート
%run ./lib/lib.ipynb

# 生データの入ったCSVファイルの保持されたディレクトリ名を格納している変数
csvDirPath = "./csv_files/"

# NPBのベンチマーク名のリスト
benchmarkNames = ["cg", "ep", "ft", "is", "lu", "mg"]

# LULESH ベンチマークプログラムのプロセス数・問題サイズ・イテレーション数
lulesh_processes: list[int] = [8, 27, 64, 125, 216, 343, 512]
lulesh_iterations: list[int] = [8, 16, 32, 64, 128, 256]
lulesh_sizes: list[int] = [16, 24, 32, 48, 64, 128]

DEBUG:__main__:hello


# 重み付き平均MAPEの算出@CG

In [16]:
date: str = "2022年8月26日"

input_list_process: list[int] = [2, 4, 8, 16, 32, 64, 128]
target_list_process: list[int] = [256]

cg_input_list_na: list[int] = [14000, 30000, 75000, 100000]
cg_input_list_nonzer: list[int] = [11, 12, 13, 14, 15, 18]
cg_input_list_niter: list[int] = [15, 30, 75, 90]
cg_input_list_shift: list[int] = [20, 40, 60, 80, 110]

cg_target_list_na: list[int] = [1500000]
cg_target_list_nonzer: list[int] = [21]
cg_target_list_niter: list[int] = [100]
cg_target_list_shift: list[int] = [200]

input_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=input_list_process,
    list_na=cg_input_list_na,
    list_nonzer=cg_input_list_nonzer,
    list_niter=cg_input_list_niter,
    list_shift=cg_input_list_shift,
    csvDir=csvDirPath,
)
input_rawDF_cg = input_rawDF_cg.rename(columns={"Name": "functionName"})

target_rawDF_cg: pd.DataFrame = return_rawDF_cg(
    list_process=target_list_process,
    list_na=cg_target_list_na,
    list_nonzer=cg_target_list_nonzer,
    list_niter=cg_target_list_niter,
    list_shift=cg_target_list_shift,
    csvDir=csvDirPath,
)
target_rawDF_cg = target_rawDF_cg.rename(columns={"Name": "functionName"})

cg_list_exp: list[str] = ["process", "na", "nonzer", "niter", "shift"]

list_res: list[str] = ["#Call"]

function_names: list[str] = sorted(list(set(input_rawDF_cg["functionName"])))

/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer18_niter30_shift60_process4.csv is empty.
  warnings.warn(f"{filePath} is empty.")
/tmp/ipykernel_1356/3490495865.py:46: UserWarning: ./csv_files/cg_na75000_nonzer15_niter90_shift80_process8.csv is empty.
  warnings.warn(f"{filePath} is empty.")


In [17]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    # "modelLinearSumOf2elementCombinationWithSquared",
    # "modelLinearSumOf2elementCombinationWithCubed",
    # "modelSquareRootOfProcess",
    # "modelSquareRootTimesOtherElems",
    # "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_before: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_before={retNum_cg_before}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_before=7.827486063247428


In [18]:
list_series: list[pd.Series] = []

list_modelName: list[str] = [
    "modelIp",
    "modelLog",
    "modelLinAndIp",
    "modelLinAndLog",
    "modelIpAndLin",
    "modelIpAndLog",
    "modelLogAndLin",
    "modelLogAndIp",
    "modelProcessDividedByProblemSize",
    "modelProblemSizeDividedByProcess",
    "modelLinearSumOf2elementCombination",
    "modelLinearSumOfElementCombinations",
    "modelLinearSumOf2elementCombinationWithSquared",
    "modelLinearSumOf2elementCombinationWithCubed",
    "modelSquareRootOfProcess",
    "modelSquareRootTimesOtherElems",
    "modelObeyOneParameter",
    "modelLin"
    # "modelBasicTree",
]

for functionName in function_names:
    rawDF_per_function_train: pd.DataFrame = input_rawDF_cg[
        input_rawDF_cg["functionName"] == functionName
    ]
    rawDF_per_function_test: pd.DataFrame = target_rawDF_cg[
        target_rawDF_cg["functionName"] == functionName
    ]

    bestModelDict: dict = return_bestModelObject(
        inputDF=rawDF_per_function_train,
        list_expVar=cg_list_exp,
        list_resVar=list_res,
        list_modelName=list_modelName,
    )

    bestModel = bestModelDict["object"]

    if bestModel is None:
        print(f"bestModelDict =\n{bestModelDict}")

    predicted = float(
        np.array(bestModel.predict(inputDF=rawDF_per_function_test[cg_list_exp]))
    )
    _call: float = float(rawDF_per_function_test.iloc[0][list_res[0]])
    _MAPE: float = float(returnMapeScore(l1=[_call], l2=[predicted]))
    _series: pd.Series = pd.Series(
        {
            "functionName": functionName,
            "call": _call,
            "MAPE": _MAPE,
            "predicted_call": predicted,
        }
    )
    list_series.append(_series)

df_toCalcWeightedMAPE: pd.DataFrame = pd.concat(list_series, axis=1).T

retNum_cg_after: float = calcWeightedMAPEscore(
    inputDF=df_toCalcWeightedMAPE,
    inputColumnDict={"funcName": "functionName", "call": "call", "MAPE": "MAPE"},
)

print(f"retNum_cg_after={retNum_cg_after}")

/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
/usr/local/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:881: OptimizeWarning: Covariance of the parameters could not be estima

retNum_cg_after=7.827486063247428
